In [6]:
import pandas as pd
from collections import Counter

def load_data(path):
    # Load the historical disasters dataset without nested parse_dates
    df = pd.read_csv(path)

    # Combine separate year, month, day columns into a single datetime
    df['StartDate'] = pd.to_datetime(
        dict(
            year=df['Start Year'],
            month=df['Start Month'],
            day=df['Start Day']
        ),
        errors='coerce'
    )
    # Drop rows where date conversion failed
    df = df.dropna(subset=['StartDate'])

    # Extract month and day for pattern matching
    df['Month'] = df['StartDate'].dt.month
    df['Day'] = df['StartDate'].dt.day
    return df

# Predefined mitigation strategies for common disaster types
MITIGATION = {
    'Drought': "Implement water conservation, drought-resistant crops, and efficient irrigation.",
    'Flood': "Construct levees, improve drainage, early warning systems, and community drills.",
    'Storm': "Strengthen infrastructure, establish shelters, and deploy meteorological monitoring.",
    'Earthquake': "Enforce building codes, retrofit structures, and develop evacuation plans.",
    'Wildfire': "Create firebreaks, controlled burns, and public education on fire safety.",
    'Eruption': "Set up exclusion zones, constant volcano monitoring, and evacuation protocols.",
    'Tropical Cyclone': "Harden coastal defenses, evacuation routes, and storm surge barriers.",
    'Heat Wave': "Open cooling centers, distribute water supplies, and public advisories."
}

def predict_disaster(df, date_str, location, top_n=3):
    """
    Predict likely disasters at a given date and location based on historical frequency,
    list previous events on that date, and suggest mitigation strategies.
    """
    # Parse user input date
    date = pd.to_datetime(date_str)
    m, d = date.month, date.day

    # Filter historical events by month/day and location
    mask_date = (df['Month'] == m) & (df['Day'] == d)
    mask_loc = (
        df['Country'].str.contains(location, case=False, na=False)
        | df['Region'].str.contains(location, case=False, na=False)
    )

    historical = df[mask_date & mask_loc]

    if historical.empty:
        print(f"No historical events found for {location} on {date_str}.")
        return

    # Count disaster types
    disasters = historical['Disaster Type'].fillna('Unknown')
    counts = Counter(disasters)
    top = counts.most_common(top_n)

    # Output predictions
    print(f"Top {top_n} predicted disasters for {location} on {date_str} based on historical data:")
    for dtype, freq in top:
        strategy = MITIGATION.get(dtype, "No specific mitigation strategy defined.")
        print(f"- {dtype} (occurred {freq} times): {strategy}")

    # Show previous events details
    print("\nPrevious events on this date:")
    cols = ['StartDate', 'Disaster Type', 'Event Name', 'Total Deaths', "Total Damages ('000 US$)"]
    print(historical[cols])

if __name__ == "__main__":
    # Hardcoded input for demonstration
    DATA_PATH = '/content/1900_2021_DISASTERS.xlsx - emdat data.csv'
    df = load_data(DATA_PATH)

    # Example: predict for August 15 in India
    predict_disaster(df, '2021-08-15', 'India')


Top 3 predicted disasters for India on 2021-08-15 based on historical data:
- Flood (occurred 3 times): Construct levees, improve drainage, early warning systems, and community drills.
- Earthquake (occurred 2 times): Enforce building codes, retrofit structures, and develop evacuation plans.
- Storm (occurred 1 times): Strengthen infrastructure, establish shelters, and deploy meteorological monitoring.

Previous events on this date:
       StartDate Disaster Type Event Name  Total Deaths  \
192   1950-08-15    Earthquake        NaN        1500.0   
669   1974-08-15         Storm        NaN          20.0   
4412  1991-08-15         Flood        NaN          88.0   
12134 2011-08-15         Flood        NaN           7.0   
12135 2011-08-15         Flood        NaN         204.0   
15413 1966-08-15    Earthquake        NaN          15.0   

       Total Damages ('000 US$)  
192                     20000.0  
669                         NaN  
4412                        NaN  
12134        

In [10]:
import pandas as pd
from collections import Counter
import requests
from geopy.distance import geodesic

# Radius (km) around user location to consider historical events
SEARCH_RADIUS_KM = 100

# Predefined mitigation strategies for common disaster types
MITIGATION = {
    'Drought': "Implement water conservation, drought-resistant crops, and efficient irrigation.",
    'Flood': "Construct levees, improve drainage, early warning systems, and community drills.",
    'Storm': "Strengthen infrastructure, establish shelters, and deploy meteorological monitoring.",
    'Earthquake': "Enforce building codes, retrofit structures, and develop evacuation plans.",
    'Wildfire': "Create firebreaks, controlled burns, and public education on fire safety.",
    'Eruption': "Set up exclusion zones, constant volcano monitoring, and evacuation protocols.",
    'Tropical Cyclone': "Harden coastal defenses, evacuation routes, and storm surge barriers.",
    'Heat Wave': "Open cooling centers, distribute water supplies, and public advisories."
}


def load_data(path):
    """
    Load and preprocess historical disasters CSV.
    """
    df = pd.read_csv(path)
    # Build StartDate, Month, Day
    df['StartDate'] = pd.to_datetime(
        dict(
            year=df['Start Year'],
            month=df['Start Month'],
            day=df['Start Day']
        ),
        errors='coerce'
    )
    df.dropna(subset=['StartDate'], inplace=True)
    df['Month'] = df['StartDate'].dt.month
    df['Day'] = df['StartDate'].dt.day
    return df


def fetch_current_weather(lat, lon):
    """
    Fetch current weather without API key using wttr.in.
    Returns temperature (°C), humidity (%), and a brief description.
    """
    url = f"http://wttr.in/{lat},{lon}?format=j1"
    resp = requests.get(url)
    data = resp.json()
    current = data['current_condition'][0]
    return {
        'temp': current['temp_C'],
        'humidity': current['humidity'],
        'weather_main': current['weatherDesc'][0]['value']
    }


def predict_disaster(df, target_date, lat, lon, top_n=3):
    """
    Predict likely disasters for a specific date and city location,
    based on historical frequency within SEARCH_RADIUS_KM of (lat,lon)
    and current weather conditions.
    """
    # Historical filter: month/day match
    m, d = target_date.month, target_date.day
    df_date = df[(df['Month'] == m) & (df['Day'] == d)]

    # Spatial filter: within radius
    def within_radius(row):
        try:
            return geodesic((lat, lon), (row['Latitude'], row['Longitude'])).km <= SEARCH_RADIUS_KM
        except Exception:
            return False

    df_spatial = df_date[df_date.apply(within_radius, axis=1)]

    # Count disaster types
    counts = Counter(df_spatial['Disaster Type'].fillna('Unknown'))
    top = counts.most_common(top_n)

    # Fetch current weather
    weather = fetch_current_weather(lat, lon)

    # Output
    print(f"--- Disaster Prediction for {target_date.date()} at location ({lat}, {lon}) ---")
    print(f"Current weather: {weather['weather_main']}, {weather['temp']}°C, humidity {weather['humidity']}%\n")
    print(f"Top {top_n} historical disaster types within {SEARCH_RADIUS_KM} km on this date:")
    for dtype, freq in top:
        strat = MITIGATION.get(dtype, "No specific mitigation defined.")
        print(f"- {dtype} (seen {freq} times): {strat}")

    # Show previous events
    if not df_spatial.empty:
        cols = ['StartDate', 'Disaster Type', 'Event Name', 'Total Deaths', "Total Damages ('000 US$)"]
        print("\nPrevious events details:")
        print(df_spatial[cols].to_string(index=False))
    else:
        print("\nNo historical events found in this area/date.")


if __name__ == '__main__':
    # Example inputs for Mumbai, India
    date_input = '2025-12-01'
    lat = 19.0760   # Mumbai latitude
    lon = 72.8777   # Mumbai longitude

    # Load data and run prediction
    DATA_PATH = '/content/1900_2021_DISASTERS.xlsx - emdat data.csv'
    df = load_data(DATA_PATH)
    target_date = pd.to_datetime(date_input)

    predict_disaster(df, target_date, lat, lon)


/usr/local/lib/python3.11/dist-packages/geopy/point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


--- Disaster Prediction for 2025-12-01 at location (19.076, 72.8777) ---
Current weather: Sunny, 29°C, humidity 56%

Top 3 historical disaster types within 100 km on this date:

No historical events found in this area/date.


In [12]:
import pandas as pd
from collections import Counter
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Radius (km) around user location to consider historical events
SEARCH_RADIUS_KM = 100

# Predefined mitigation strategies for common disaster types
MITIGATION = {
    'Drought': "Implement water conservation, drought-resistant crops, and efficient irrigation.",
    'Flood': "Construct levees, improve drainage, early warning systems, and community drills.",
    'Storm': "Strengthen infrastructure, establish shelters, and deploy meteorological monitoring.",
    'Earthquake': "Enforce building codes, retrofit structures, and develop evacuation plans.",
    'Wildfire': "Create firebreaks, controlled burns, and public education on fire safety.",
    'Eruption': "Set up exclusion zones, constant volcano monitoring, and evacuation protocols.",
    'Tropical Cyclone': "Harden coastal defenses, evacuation routes, and storm surge barriers.",
    'Heat Wave': "Open cooling centers, distribute water supplies, and public advisories."
}

def load_data(path):
    """
    Load and preprocess historical disasters CSV.
    """
    df = pd.read_csv(path)
    # Build StartDate, Month, Day
    df['StartDate'] = pd.to_datetime(
        dict(
            year=df['Start Year'],
            month=df['Start Month'],
            day=df['Start Day']
        ),
        errors='coerce'
    )
    df.dropna(subset=['StartDate'], inplace=True)
    df['Month'] = df['StartDate'].dt.month
    df['Day'] = df['StartDate'].dt.day
    return df


def fetch_current_weather(lat, lon):
    """
    Fetch current weather without API key using wttr.in.
    Returns temperature (°C), humidity (%), and a brief description.
    """
    url = f"http://wttr.in/{lat},{lon}?format=j1"
    resp = requests.get(url)
    data = resp.json()
    current = data['current_condition'][0]
    return {
        'temp': current['temp_C'],
        'humidity': current['humidity'],
        'weather_main': current['weatherDesc'][0]['value']
    }


def get_city_name(lat, lon):
    """
    Reverse geocode latitude and longitude to get the city name.
    """
    geolocator = Nominatim(user_agent="disaster_predictor")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location and 'address' in location.raw:
        addr = location.raw['address']
        # Try common keys for city-like names
        for key in ['city', 'town', 'village', 'municipality', 'state_district', 'state']:
            if key in addr:
                return addr[key]
    return 'Unknown Location'


def predict_disaster(df, target_date, lat, lon, top_n=3):
    """
    Predict likely disasters for a specific date and city location,
    based on historical frequency within SEARCH_RADIUS_KM of (lat,lon)
    and current weather conditions.
    """
    # Determine city name
    city = get_city_name(lat, lon)

    # Historical filter: month/day match
    m, d = target_date.month, target_date.day
    df_date = df[(df['Month'] == m) & (df['Day'] == d)]

    # Spatial filter: within radius
    def within_radius(row):
        try:
            return geodesic((lat, lon), (row['Latitude'], row['Longitude'])).km <= SEARCH_RADIUS_KM
        except Exception:
            return False

    df_spatial = df_date[df_date.apply(within_radius, axis=1)]

    # Count disaster types
    counts = Counter(df_spatial['Disaster Type'].fillna('Unknown'))
    top = counts.most_common(top_n)

    # Fetch current weather
    weather = fetch_current_weather(lat, lon)

    # Output
    print(f"--- Disaster Prediction for {target_date.date()} in {city} ({lat}, {lon}) ---")
    print(f"Current weather: {weather['weather_main']}, {weather['temp']}°C, humidity {weather['humidity']}%\n")
    print(f"Top {top_n} historical disaster types within {SEARCH_RADIUS_KM} km on this date:")
    for dtype, freq in top:
        strat = MITIGATION.get(dtype, "No specific mitigation defined.")
        print(f"- {dtype} (seen {freq} times): {strat}")

    # Show previous events
    if not df_spatial.empty:
        cols = ['StartDate', 'Disaster Type', 'Event Name', 'Total Deaths', "Total Damages ('000 US$)"]
        print("\nPrevious events details:")
        print(df_spatial[cols].to_string(index=False))
    else:
        print("\nNo historical events found in this area/date.")


if __name__ == '__main__':
    # Example inputs for Chennai, India (historic floods)
    date_input = '2015-12-01'
    lat = 13.0827  # Chennai latitude
    lon = 80.2707  # Chennai longitude

    # Load data and run prediction
    DATA_PATH = '/content/1900_2021_DISASTERS.xlsx - emdat data.csv'
    df = load_data(DATA_PATH)
    target_date = pd.to_datetime(date_input)

    predict_disaster(df, target_date, lat, lon)

/usr/local/lib/python3.11/dist-packages/geopy/point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


--- Disaster Prediction for 2015-12-01 in Chennai (13.0827, 80.2707) ---
Current weather: Partly cloudy, 34°C, humidity 67%

Top 3 historical disaster types within 100 km on this date:

No historical events found in this area/date.


In [13]:
import pandas as pd
from collections import Counter
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Radius (km) around user location to consider historical events
SEARCH_RADIUS_KM = 100

# Predefined mitigation strategies for common disaster types
MITIGATION = {
    'Drought': "Implement water conservation, drought-resistant crops, and efficient irrigation.",
    'Flood': "Construct levees, improve drainage, early warning systems, and community drills.",
    'Storm': "Strengthen infrastructure, establish shelters, and deploy meteorological monitoring.",
    'Earthquake': "Enforce building codes, retrofit structures, and develop evacuation plans.",
    'Wildfire': "Create firebreaks, controlled burns, and public education on fire safety.",
    'Eruption': "Set up exclusion zones, constant volcano monitoring, and evacuation protocols.",
    'Tropical Cyclone': "Harden coastal defenses, evacuation routes, and storm surge barriers.",
    'Heat Wave': "Open cooling centers, distribute water supplies, and public advisories."
}

def load_data(path):
    """
    Load and preprocess historical disasters CSV.
    """
    df = pd.read_csv(path)
    # Build StartDate, Month, Day
    df['StartDate'] = pd.to_datetime(
        dict(
            year=df['Start Year'],
            month=df['Start Month'],
            day=df['Start Day']
        ),
        errors='coerce'
    )
    df.dropna(subset=['StartDate'], inplace=True)
    df['Month'] = df['StartDate'].dt.month
    df['Day'] = df['StartDate'].dt.day
    return df


def fetch_current_weather(lat, lon):
    """
    Fetch current weather without API key using wttr.in.
    Returns temperature (°C), humidity (%), and a brief description.
    """
    url = f"http://wttr.in/{lat},{lon}?format=j1"
    resp = requests.get(url)
    data = resp.json()
    current = data['current_condition'][0]
    return {
        'temp': current['temp_C'],
        'humidity': current['humidity'],
        'weather_main': current['weatherDesc'][0]['value']
    }


def get_city_name(lat, lon):
    """
    Reverse geocode latitude and longitude to get the city name.
    """
    geolocator = Nominatim(user_agent="disaster_predictor")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location and 'address' in location.raw:
        addr = location.raw['address']
        for key in ['city', 'town', 'village', 'municipality', 'state_district', 'state']:
            if key in addr:
                return addr[key]
    return 'Unknown Location'


def predict_disaster(df, target_date, lat, lon, top_n=3):
    """
    Predict likely disasters for a specific date and city location,
    based on historical frequency within SEARCH_RADIUS_KM of (lat,lon)
    and current weather conditions.
    """
    city = get_city_name(lat, lon)

    # Historical filter: month/day match
    m, d = target_date.month, target_date.day
    df_date = df[(df['Month'] == m) & (df['Day'] == d)]

    # Spatial filter: within radius
    def within_radius(row):
        try:
            return geodesic((lat, lon), (row['Latitude'], row['Longitude'])).km <= SEARCH_RADIUS_KM
        except:
            return False

    df_spatial = df_date[df_date.apply(within_radius, axis=1)]

    # Count disaster types
    counts = Counter(df_spatial['Disaster Type'].fillna('Unknown'))
    top = counts.most_common(top_n)

    # Fetch current weather
    weather = fetch_current_weather(lat, lon)

    # Output
    print(f"--- Disaster Prediction for {target_date.date()} in {city} ({lat}, {lon}) ---")
    print(f"Current weather: {weather['weather_main']}, {weather['temp']}°C, humidity {weather['humidity']}%\n")
    print(f"Top {top_n} historical disaster types within {SEARCH_RADIUS_KM} km on this date:")
    for dtype, freq in top:
        strat = MITIGATION.get(dtype, "No specific mitigation defined.")
        print(f"- {dtype} (seen {freq} times): {strat}")

    # Show previous events
    if not df_spatial.empty:
        cols = ['StartDate', 'Disaster Type', 'Event Name', 'Total Deaths', "Total Damages ('000 US$)"]
        print("\nPrevious events details:")
        print(df_spatial[cols].to_string(index=False))
    else:
        print("\nNo historical events found in this area/date.")


if __name__ == '__main__':
    # Example inputs for Mumbai, India (2005 flood event)
    date_input = '2005-10-26'
    lat = 19.0760   # Mumbai latitude
    lon = 72.8777   # Mumbai longitude

    # Load data and run prediction
    DATA_PATH = '/content/1900_2021_DISASTERS.xlsx - emdat data.csv'
    df = load_data(DATA_PATH)
    target_date = pd.to_datetime(date_input)

    predict_disaster(df, target_date, lat, lon)


--- Disaster Prediction for 2005-10-26 in Mumbai Suburban (19.076, 72.8777) ---
Current weather: Sunny, 29°C, humidity 56%

Top 3 historical disaster types within 100 km on this date:

No historical events found in this area/date.


In [16]:
import pandas as pd
from collections import Counter
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Radius (km) around user location to consider historical events
SEARCH_RADIUS_KM = 100

# Predefined mitigation strategies for common disaster types
MITIGATION = {
    'Drought': "Implement water conservation, drought-resistant crops, and efficient irrigation.",
    'Flood': "Construct levees, improve drainage, early warning systems, and community drills.",
    'Storm': "Strengthen infrastructure, establish shelters, and deploy meteorological monitoring.",
    'Earthquake': "Enforce building codes, retrofit structures, and develop evacuation plans.",
    'Wildfire': "Create firebreaks, controlled burns, and public education on fire safety.",
    'Eruption': "Set up exclusion zones, constant volcano monitoring, and evacuation protocols.",
    'Tropical Cyclone': "Harden coastal defenses, evacuation routes, and storm surge barriers.",
    'Heat Wave': "Open cooling centers, distribute water supplies, and public advisories."
}

def load_data(path):
    """
    Load and preprocess historical disasters CSV.
    """
    df = pd.read_csv(path)
    df['StartDate'] = pd.to_datetime(
        dict(
            year=df['Start Year'],
            month=df['Start Month'],
            day=df['Start Day']
        ),
        errors='coerce'
    )
    df.dropna(subset=['StartDate'], inplace=True)
    df['Month'] = df['StartDate'].dt.month
    df['Day'] = df['StartDate'].dt.day
    return df


def fetch_current_weather(lat, lon):
    """
    Fetch current weather without API key using wttr.in.
    Returns temperature (°C), humidity (%), and a brief description.
    """
    url = f"http://wttr.in/{lat},{lon}?format=j1"
    resp = requests.get(url)
    data = resp.json()
    current = data['current_condition'][0]
    return {
        'temp': current['temp_C'],
        'humidity': current['humidity'],
        'weather_main': current['weatherDesc'][0]['value']
    }


def get_city_name(lat, lon):
    """
    Reverse geocode latitude and longitude to get the city name.
    """
    geolocator = Nominatim(user_agent="disaster_predictor")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location and 'address' in location.raw:
        addr = location.raw['address']
        for key in ['city', 'town', 'village', 'municipality', 'state_district', 'state']:
            if key in addr:
                return addr[key]
    return 'Unknown Location'


def predict_disaster(df, target_date, lat, lon, top_n=3):
    """
    Predict likely disasters for a specific date and city location,
    based on historical frequency within SEARCH_RADIUS_KM of (lat,lon)
    and current weather conditions.
    """
    city = get_city_name(lat, lon)

    # Historical filter: month/day match
    m, d = target_date.month, target_date.day
    df_date = df[(df['Month'] == m) & (df['Day'] == d)]

    # Spatial filter: within radius
    def within_radius(row):
        try:
            return geodesic((lat, lon), (row['Latitude'], row['Longitude'])).km <= SEARCH_RADIUS_KM
        except:
            return False

    df_spatial = df_date[df_date.apply(within_radius, axis=1)]

    # Count disaster types
    counts = Counter(df_spatial['Disaster Type'].fillna('Unknown'))
    top = counts.most_common(top_n)

    # Fetch current weather
    weather = fetch_current_weather(lat, lon)

    # Output
    print(f"--- Disaster Prediction for {target_date.date()} in {city} ({lat}, {lon}) ---")
    print(f"Current weather: {weather['weather_main']}, {weather['temp']}°C, humidity {weather['humidity']}%\n")
    print(f"Top {top_n} historical disaster types within {SEARCH_RADIUS_KM} km on this date:")
    for dtype, freq in top:
        strat = MITIGATION.get(dtype, "No specific mitigation defined.")
        print(f"- {dtype} (seen {freq} times): {strat}")

    # Show previous events or placeholder
    if not df_spatial.empty:
        cols = ['StartDate', 'Disaster Type', 'Event Name', 'Total Deaths', "Total Damages ('000 US$)"]
        print("\nPrevious events details:")
        print(df_spatial[cols].to_string(index=False))
    else:
        print("\nNo historical events found in this area/date. Displaying placeholder sample:")
        sample = pd.DataFrame([{
            'StartDate': target_date,
            'Disaster Type': 'Sample Disaster',
            'Event Name': 'Placeholder event due to lack of historical data',
            'Total Deaths': 0,
            "Total Damages ('000 US$)": 0
        }])
        cols = ['StartDate', 'Disaster Type', 'Event Name', 'Total Deaths', "Total Damages ('000 US$)"]
        print(sample[cols].to_string(index=False))


if __name__ == '__main__':
    # Example inputs for Mumbai, India (2005 flood event)
    date_input = '2005-10-26'
    lat = 19.0760   # Mumbai latitude
    lon = 72.8777   # Mumbai longitude

    # Load data and run prediction
    DATA_PATH = '/content/1900_2021_DISASTERS.xlsx - emdat data.csv'
    df = load_data(DATA_PATH)
    target_date = pd.to_datetime(date_input)

    predict_disaster(df, target_date, lat, lon)


--- Disaster Prediction for 2005-10-26 in Mumbai Suburban (19.076, 72.8777) ---
Current weather: Sunny, 29°C, humidity 56%

Top 3 historical disaster types within 100 km on this date:

No historical events found in this area/date. Displaying placeholder sample:
 StartDate   Disaster Type                                       Event Name  Total Deaths  Total Damages ('000 US$)
2005-10-26 Sample Disaster Placeholder event due to lack of historical data             0                         0


In [19]:
# Disaster Predictor for Indian Cities
# -------------------------------------
# Predicts likely natural disasters for a given date and city coordinates in India,
# using historical data (with fallback sample) and current weather.

import pandas as pd
from collections import Counter
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Radius (km) around user location to consider historical events
SEARCH_RADIUS_KM = 100

# Sample fallback data if CSV not found or malformed
def sample_data():
    data = [
        {
            'City': 'Mumbai', 'Latitude': 19.0760, 'Longitude': 72.8777,
            'Disaster_Type': 'Flood', 'Year': 2005, 'Month': 10, 'Day': 26,
            'Deaths': 1098, 'Damages_USD_000': 30000
        },
        {
            'City': 'Chennai', 'Latitude': 13.0827, 'Longitude': 80.2707,
            'Disaster_Type': 'Cyclone', 'Year': 2015, 'Month': 12, 'Day': 2,
            'Deaths': 13, 'Damages_USD_000': 15000
        },
        {
            'City': 'Delhi', 'Latitude': 28.7041, 'Longitude': 77.1025,
            'Disaster_Type': 'Heat Wave', 'Year': 2019, 'Month': 6, 'Day': 14,
            'Deaths': 40, 'Damages_USD_000': 5000
        }
    ]
    df = pd.DataFrame(data)
    df['StartDate'] = pd.to_datetime(
        dict(year=df['Year'], month=df['Month'], day=df['Day'])
    )
    df['Month'] = df['StartDate'].dt.month
    df['Day'] = df['StartDate'].dt.day
    return df

# Predefined mitigation strategies for disaster types
MITIGATION = {
    'Drought': "Implement water conservation, drought-resistant crops, and efficient irrigation.",
    'Flood': "Construct levees, improve drainage, early warning systems, and community drills.",
    'Cyclone': "Issue timely warnings, reinforce shelters, and clear drainage.",
    'Heat Wave': "Open cooling centers, distribute water, and issue advisories.",
    'Earthquake': "Enforce building codes, retrofit structures, and plan evacuations."
}

def load_data(path):
    """
    Attempt to load India-specific disasters CSV with columns:
    City, Latitude, Longitude, Disaster_Type, Year, Month, Day, Deaths, Damages_USD_000
    Fallback to sample_data() if load fails or columns missing.
    """
    try:
        df = pd.read_csv(path)
        # Check required columns
        req = {'City','Latitude','Longitude','Disaster_Type','Year','Month','Day','Deaths','Damages_USD_000'}
        if not req.issubset(df.columns):
            raise ValueError("Missing columns")
        # Build StartDate
        df['StartDate'] = pd.to_datetime(
            dict(year=df['Year'], month=df['Month'], day=df['Day']),
            errors='coerce'
        )
        df.dropna(subset=['StartDate','Latitude','Longitude'], inplace=True)
        df['Month'] = df['StartDate'].dt.month
        df['Day'] = df['StartDate'].dt.day
        return df
    except Exception:
        print("Could not load CSV; using sample data.")
        return sample_data()


def fetch_current_weather(lat, lon):
    """
    Fetch current weather using wttr.in (no API key).
    """
    url = f"http://wttr.in/{lat},{lon}?format=j1"
    try:
        data = requests.get(url).json()
        cur = data['current_condition'][0]
        return {
            'temp': cur['temp_C'], 'humidity': cur['humidity'],
            'weather_main': cur['weatherDesc'][0]['value']
        }
    except Exception:
        return {'temp': 'N/A', 'humidity': 'N/A', 'weather_main': 'Unknown'}


def predict_disaster(df, date_str, lat, lon, top_n=3):
    """
    Display prediction for date_str (YYYY-MM-DD) and lat/lon.
    """
    target_date = pd.to_datetime(date_str, errors='coerce')
    if pd.isna(target_date):
        print("Invalid date format.")
        return

    # Reverse geocode city
    geolocator = Nominatim(user_agent="disaster_predictor")
    try:
        loc = geolocator.reverse((lat, lon), exactly_one=True)
        addr = loc.raw.get('address', {})
        city = addr.get('city') or addr.get('town') or addr.get('state') or 'Unknown'
    except Exception:
        city = 'Unknown'

    # Filter by month/day
    m, d = target_date.month, target_date.day
    df_date = df[(df['Month']==m)&(df['Day']==d)]

    # Spatial filter
    df_spatial = df_date[df_date.apply(
        lambda r: geodesic((lat, lon),(r['Latitude'],r['Longitude'])).km <= SEARCH_RADIUS_KM,
        axis=1
    )]

    # Count and top_n
    counts = Counter(df_spatial['Disaster_Type'])
    top = counts.most_common(top_n)

    # Current weather
    weather = fetch_current_weather(lat, lon)

    # Output
    print(f"--- Prediction for {target_date.date()} in {city} ({lat}, {lon}) ---")
    print(f"Current weather: {weather['weather_main']}, {weather['temp']}°C, humidity {weather['humidity']}%\n")
    print(f"Top {top_n} historical disasters within {SEARCH_RADIUS_KM}km on this date:")
    if top:
        for dt, freq in top:
            print(f"- {dt} (seen {freq} times): {MITIGATION.get(dt,'No strategy')}")
    else:
        print("No historical events; showing sample placeholder:")
        dt, freq = 'Sample Disaster', 0
        print(f"- {dt} (seen {freq} times): {MITIGATION.get(dt,'No strategy')}" )

    # Detail table
    cols = ['StartDate','Disaster_Type','City','Deaths','Damages_USD_000']
    if not df_spatial.empty:
        print("\nPast events:")
        print(df_spatial[cols].to_string(index=False))
    else:
        print("\nPlaceholder event:")
        placeholder = pd.DataFrame([{ 'StartDate':target_date,
                                      'Disaster_Type':'Sample Disaster',
                                      'City':city, 'Deaths':0, 'Damages_USD_000':0 }])
        print(placeholder[cols].to_string(index=False))

if __name__=='__main__':
    # Hardcoded example for Mumbai, India
    date_str = '2025-12-07'  # YYYY-MM-DD format
    lat = 19.0760           # Mumbai latitude
    lon = 72.8777           # Mumbai longitude

    df = load_data('/content/disasterIND.csv')
    predict_disaster(df, date_str, lat, lon)


Could not load CSV; using sample data.
--- Prediction for 2025-12-07 in Mumbai Suburban (19.076, 72.8777) ---
Current weather: Sunny, 29°C, humidity 56%

Top 3 historical disasters within 100km on this date:
No historical events; showing sample placeholder:
- Sample Disaster (seen 0 times): No strategy

Placeholder event:
 StartDate   Disaster_Type            City  Deaths  Damages_USD_000
2025-12-07 Sample Disaster Mumbai Suburban       0                0
